In [52]:
import pandas as pd
import requests
from plyer import notification
from datetime import datetime
import sqlite3
from tqdm import tqdm

In [53]:
def alerta(nivel, base, etapa):
    if nivel == 1:
        titulo = "Alerta Baixo"
    elif nivel == 2:
        titulo = "Alerta Médio"
    elif nivel == 3:
        titulo = "Alerta Alto"
    else:
        titulo = "Alerta Desconhecido"

    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    mensagem = f"Falha no carregamento da base {base} na etapa {etapa}\nData: {data_atual}"

    notification.notify(
        title=titulo,
        message=mensagem,
        timeout=20
    )

In [54]:
def salva_bd(df, nome_tabela):
    conn = sqlite3.connect('data_pipeline.db')
    df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    conn.close()

def carrega_bd(nome_tabela):
    conn = sqlite3.connect('data_pipeline.db')
    query = f"SELECT * FROM {nome_tabela}"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [55]:
def get_json_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        alerta(3, url, "Requisição", response.text)
        raise Exception(f"Erro na requisição: {response.status_code}")

def get_bancos():
    url = "https://brasilapi.com.br/api/banks/v1"
    data_json = get_json_api(url)
    df_bancos = pd.DataFrame(data_json)
    return df_bancos

def get_pix_participants():
    url = "https://brasilapi.com.br/api/pix/v1/participants"
    data_json = get_json_api(url)
    df_pix = pd.DataFrame(data_json)
    return df_pix

def get_corretoras():
    url = "https://brasilapi.com.br/api/cvm/corretoras/v1"
    data_json = get_json_api(url)
    df_corretoras = pd.DataFrame(data_json)
    return df_corretoras

In [56]:
def etapa_extracao():
    try:
        df_bancos = get_bancos()
        salva_bd(df_bancos, "bancos")
        
    except Exception as e:
        alerta(3, "bancos", "EXTRACAO", str(e))

    try:
        df_pix = get_pix_participants()
        salva_bd(df_pix, "pix_participants")
    except Exception as e:
        alerta(3, "pix_participants", "EXTRACAO", str(e))

    try:
        df_corretoras = get_corretoras()
        salva_bd(df_corretoras, "corretoras")
    except Exception as e:
        alerta(3, "corretoras", "EXTRACAO", str(e))

In [57]:
get_corretoras()

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL DTVM,CANCELADA,complianceregulatorio@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,1989-12-31,5995252.29,329,1990-06-12,1986-07-08
3,44527444000155,CORRETORAS,ABN AMRO CLEARING CORRETORA DE T-TULOS E VALOR...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",2023-12-31,11122779.75,3794,2022-06-20,2022-06-20
4,33819590000138,CORRETORAS,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,,5095755,20040040,BRASIL,RJ,RIO DE JANEIRO,CENTRO,,"PRAÇA OLAVO BILAC, 28/ SL. 608",2000-12-31,291862.38,1112,2001-04-11,1968-04-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,01170892000131,CORRETORAS,VOTORANTIM CTVM LTDA,VOTORANTIM,CANCELADA,demandasregulatorias@bancovotorantim.com.br,51711894,4794000,BRASIL,SP,SÃO PAULO,VL.GERTRUDES,TORRE A-14º ANDAR,"AV DAS NAÇÕES UNIDAS, 14.171",2017-12-31,266791832.77,3166,2018-08-27,1996-03-15
370,61769790000169,CORRETORAS,WALPIRES S/A CCTVM,WALPIRES S/A CCTVM,CANCELADA,andre@walpires.com.br,28087065,1452002,BRASIL,SP,SÃO PAULO,JARDIM PAULISTANO,1309 - 9ºANDAR,AVENIDA BRIGADEIRO FARIA LIMA,2017-12-31,7464373.90,1902,2019-09-24,1967-09-20
371,92875780000131,CORRETORAS,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,WARREN CORRETORA,EM FUNCIONAMENTO NORMAL,juridico@warren.com.br,32372273,90035191,,RS,PORTO ALEGRE,BOM FIM,2º ANDAR,"AVENIDA OSVALDO ARANHA, 720",2023-12-31,20672910.31,2534,1967-12-06,1967-12-06
372,24543688000114,CORRETORAS,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,PERFIL,CANCELADA,,2249022,52061540,BRASIL,PE,RECIFE,CASA FORTE,,"AV.17 DE AGOSTO,2579",1996-12-31,608260.02,2658,1997-02-21,1989-11-14


In [58]:
get_pix_participants()

,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z
...,...,...,...,...,...,...
836,95213211,"COOPERATIVA DE CRéDITO, POUPANçA E INVESTIMENT...",COOP SICREDI CAMINHO DAS ÁGUAS RS,PDCT,IDRT,2020-11-03T09:30:00.000Z
837,95424891,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI VALE DO RIO PARDO,PDCT,IDRT,2020-11-03T09:30:00.000Z
838,95594941,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI REG CENTRO RS/MG,PDCT,IDRT,2020-11-03T09:30:00.000Z
839,97259253,COOPERATIVA E ECONOMIA E CRÉDITO MÚTUO DOS MÉD...,CECM MÉDICOS PLANALTO MÉDIO RS - CREDISIS/CRED...,PDCT,IDRT,2021-10-04T12:53:10.567Z


In [59]:
get_bancos()

,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
...,...,...,...,...
354,92875780,WARREN CVMC LTDA,371.0,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212.0,Banco Original S.A.
356,94968518,EFX CC LTDA.,289.0,EFX CORRETORA DE CÂMBIO LTDA.
357,,NaN,NaN,NaN


In [60]:
def etapa_transformacao():
    try:
        df_bancos = carrega_bd("bancos")
        salva_bd(df_bancos, "bancos_transformados")
    except Exception as e:
        alerta(2, "bancos_transformados", "TRANSFORMACAO", str(e))

    try:
        df_pix = carrega_bd("pix_participants")
        salva_bd(df_pix, "pix_participants_transformados")
    except Exception as e:
        alerta(2, "pix_participants_transformados", "TRANSFORMACAO", str(e))

    try:
        df_corretoras = carrega_bd("corretoras")
        salva_bd(df_corretoras, "corretoras_transformadas")
    except Exception as e:
        alerta(2, "corretoras_transformadas", "TRANSFORMACAO", str(e))

    try:
        df_bancos_transformados = carrega_bd("bancos_transformados")
        df_pix_transformados = carrega_bd("pix_participants_transformados")
        
        df_bancos_transformados = df_bancos_transformados.drop(columns=["nome_reduzido", "nome"], errors='ignore')
        df_pix_transformados = df_pix_transformados.drop(columns=["nome_reduzido", "nome"], errors='ignore')
        

        df_combined = pd.merge(df_bancos_transformados, df_pix_transformados, on="ispb", suffixes=('_bancos', '_pix'))
        

        salva_bd(df_combined, "bancos_pix_combined")
    except Exception as e:
        alerta(2, "bancos_pix_combined", "TRANSFORMACAO", str(e))

In [61]:
etapa_extracao()

In [62]:
etapa_transformacao()

In [63]:
df = carrega_bd('bancos_pix_combined')
df.head()

,ispb,name,code,fullName,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00204963,CCR SEARA,430.0,COOPERATIVA DE CREDITO RURAL SEARA - CREDISEARA,PDCT,DRCT,2022-02-11T13:00:00.000Z
3,00315557,CONF NAC COOP CENTRAIS UNICRED,136.0,CONFEDERAÇÃO NACIONAL DAS COOPERATIVAS CENTRAI...,PDCT,DRCT,2020-11-03T09:30:00.000Z
4,00329598,ÍNDIGO INVESTIMENTOS DTVM LTDA.,407.0,ÍNDIGO INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS ...,PDCT,DRCT,2022-11-08T13:00:00.000Z


In [64]:
df.shape

(162, 7)